## Notebook Import

In [160]:
import pandas as pd
import numpy as np


## Constants

In [161]:
Training_Data_File='SpamData/02_Training/train-data.txt'
Test_Data_File='SpamData/02_Training/test-data.txt'

Token_Spam_Prob_File='SpamData/03_Testing/prob-spam.txt'
Token_Ham_Prob_File='SpamData/03_Testing/prob-ham.txt'
Token_All_Prob_File='SpamData/03_Testing/prob-all-tokens.txt'

Test_Feature_Matrix='SpamData/03_Testing/test-features.txt'
Test_Target_File='SpamData/03_Testing/test-target.txt'

Vocab_size=2500

### Read and Load Features from .txt File into NumPy array

In [162]:
sparse_train_data=np.loadtxt(Training_Data_File,delimiter=' ',dtype=int)
# delimiter is like seperator for data , dtype- data type
sparse_test_data=np.loadtxt(Test_Data_File,delimiter=' ',dtype=int)

In [163]:
sparse_train_data[:5]

array([[0, 0, 1, 1],
       [0, 1, 1, 4],
       [0, 2, 1, 4],
       [0, 3, 1, 6],
       [0, 4, 1, 1]])

In [164]:
print('Nr of rows in training file ',sparse_train_data.shape[0])
print('Nr of rows in test file ',sparse_test_data.shape[0])

Nr of rows in training file  266555
Nr of rows in test file  109579


In [165]:
print('Nr of emails in training file ',np.unique(sparse_train_data[:,0]).size)
print('Nr of emails in test file ',np.unique(sparse_test_data[:,0]).size)

Nr of emails in training file  4017
Nr of emails in test file  1720


## How to Create an Empty Dataframe

In [166]:
column_name=['Doc_Id','Category']+list(range(0,Vocab_size))
column_name[:5]


['Doc_Id', 'Category', 0, 1, 2]

In [167]:
len(column_name)

2502

In [168]:
index_names=np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [169]:
full_train_data=pd.DataFrame(index=index_names,columns=column_name)
full_train_data.fillna(value=0,inplace=True)

In [170]:
full_train_data.head()

,Doc_Id,Category,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### little thing used for converting sparse to full matrix

In [171]:
sparse_train_data[10:13]

array([[ 0, 12,  1,  1],
       [ 0, 13,  1,  1],
       [ 0, 14,  1,  4]])

In [172]:
sparse_train_data[10][0]

0

## Create a Full Matrix from Sparse Matrix

In [173]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    '''
    Form a Full matrix from a sparse matrix . return a pandas dataframe.
    Keywords Argument:\n
    sparse_matrix -- numpy array\n
    nr_words -- Size of Vocabulary. Total no of tokens .
    doc_idx -- pos of doc id in sparse matrix . Default : 1st column
    word_idx -- pos of word id in sparse matrix. Default : 2nd column
    cat_idx -- position of label (spam is 1 ,nonspam is 0). Default : 3rd column
    freq_idx -- position of occurence of sparse matrix . Default : 4th column
    '''
    column_name=['Doc_Id','Category']+list(range(0,Vocab_size))
    doc_id_names=np.unique(sparse_matrix[:,0])
    full_matrix=pd.DataFrame(index=doc_id_names,columns=column_name)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr=sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence= sparse_matrix[i][freq_idx]

        full_matrix.at[doc_nr,'Doc_Id']=doc_nr
        full_matrix.at[doc_nr,'Category']=label
        full_matrix.at[doc_nr,word_id]=occurence
    full_matrix.set_index('Doc_Id',inplace=True)
    return full_matrix

In [174]:
sparse_train_data.shape

(266555, 4)

In [175]:
full_train_data=make_full_matrix(sparse_train_data,Vocab_size)

In [176]:
full_train_data.head()

,Category,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
Doc_Id,,,,,,,,,,,,,,,,,,,,,
0,1,1,4,4,6,1,1,3,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,1,0,3,2,1,6,2,1,...,0,0,0,0,0,0,0,0,0,0
2,1,2,1,2,3,2,2,2,4,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,1,2,3,2,2,2,4,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,2,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Naive Bayes Model
### Calculating the Probability of Spam

In [177]:
prob_spam=sum(full_train_data['Category']==1)/full_train_data.shape[0]
# prob_spam = full_train_data.Category.sum()/full_train_data.Category.size
print("probability of spam is : ",prob_spam)
#  0.3116753796365447

probability of spam is :  0.3116753796365447


### Total Number of Words or Tokens

In [178]:
full_train_features=full_train_data.loc[:,full_train_data.columns != 'Category']


In [179]:
email_lengths=full_train_features.sum(axis=1)
email_lengths.shape

(4017,)

In [180]:
total_wc=email_lengths.sum()
total_wc

456368

### Number of Token in Spam and Ham Emails

In [181]:
spam_lengths=email_lengths[full_train_data.Category==1]
spam_lengths.shape

(1252,)

In [182]:
spam_wc=spam_lengths.sum()
spam_wc

185658

In [183]:
ham_lengths=full_train_features[full_train_data.Category==0].sum(axis=1)
ham_lengths.shape

(2765,)

In [184]:
nonspam_wc=ham_lengths.sum()
nonspam_wc

270710

In [185]:
spam_wc+nonspam_wc-total_wc

0

In [186]:
# To Find out Which email is long spam or ham?
print('Average no of word in spam email {:.0f}'.format(spam_wc/spam_lengths.shape[0]))
print('Average no of word in ham email {:.2f}'.format(nonspam_wc/ham_lengths.shape[0]))

Average no of word in spam email 148
Average no of word in ham email 97.91


## Summing the Tokens Occuring in Spam

In [187]:
train_spam_tokens=full_train_features.loc[full_train_data.Category==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
Doc_Id,,,,,,,,,,,,,,,,,,,,,
0,1,4,4,6,1,1,3,3,0,4,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,3,2,1,6,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,1,2,3,2,2,2,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1,2,3,2,2,2,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
summed_spam_tokens = train_spam_tokens.sum(axis=0)+1 # add 1 to ensure that no of token is nonzero
summed_spam_tokens.shape

(2500,)

In [189]:
summed_spam_tokens.tail()

2495    13
2496    28
2497     9
2498     3
2499    24
dtype: int64

## Summing the Tokens Occuring in Ham


In [190]:
train_ham_tokens=full_train_features.loc[full_train_data.Category==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1 # add 1 to ensure that no of token is nonzero
summed_ham_tokens.shape

(2500,)

In [191]:
summed_ham_tokens.tail()

2495    26
2496     4
2497    30
2498    34
2499     6
dtype: int64

### P(Token|Spam)- Probablity that a Token Occurs given the Email is Spam

In [192]:
prob_tokens_spam=summed_spam_tokens/(spam_wc+Vocab_size) # add vocab_size to balance sum 
prob_tokens_spam[:5]                                     # of 1 in summed_spam_tokens
                                                         

0    0.010417
1    0.004618
2    0.006532
3    0.010045
4    0.006766
dtype: float64

In [193]:
prob_tokens_spam.sum()

1.0000000000000002

### P(Token|Ham)- Probablity that a Token Occurs given the Email is NonSpam

In [194]:
prob_tokens_nonspam=summed_ham_tokens/(nonspam_wc+Vocab_size) # add vocab_size to balance sum 
prob_tokens_nonspam.sum()                                     # of 1 in summed_ham_tokens

1.0

## p(Token)- Probablity that Token Occurs

In [195]:
prob_tokens_all=full_train_features.sum(axis=0)/total_wc
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [196]:
np.savetxt(Token_Spam_Prob_File,prob_tokens_spam)
np.savetxt(Token_Ham_Prob_File,prob_tokens_nonspam)
np.savetxt(Token_All_Prob_File,prob_tokens_all)
import joblib
from sklearn.ensemble import RandomForestClassifier

# Assuming X_train and y_train are your features and labels
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
# Save the trained model to a file
joblib.dump(model, 'bias_classifier_model.pkl')


## Prepare Test Data 

In [197]:
sparse_test_data.shape

(109579, 4)

In [198]:
full_test_data=make_full_matrix(sparse_test_data,Vocab_size)

In [199]:
X_test=full_test_data.loc[:,full_test_data.columns !="Category"]
y_test=full_test_data.Category

In [200]:
np.savetxt(Test_Target_File,y_test)
np.savetxt(Test_Feature_Matrix,X_test)